In [118]:
# ALL The Imports:

import pandas as pd
import numpy as np
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, SMOTENC
import time
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from textblob import TextBlob
import json

# SKlearn imports:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, balanced_accuracy_score, RocCurveDisplay, recall_score, precision_score, recall_score, accuracy_score, ConfusionMatrixDisplay, roc_auc_score, precision_score, confusion_matrix
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor, plot_tree, DecisionTreeClassifier
from sklearn import set_config
from sklearn.naive_bayes import GaussianNB, MultinomialNB
set_config(display = 'diagram')
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, BaggingRegressor,  RandomForestRegressor

In [2]:
df = pd.read_csv('./data/titles.csv')

# Logistic regression from the title

In [14]:
X = df['title']
y = df['subreddit']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=123)

In [24]:
# using stop words
cvect_pipe = make_pipeline(CountVectorizer(stop_words = 'english', max_features = 1000),
                           LogisticRegression(max_iter=1000, penalty="none"))

cvect_pipe.fit(X_train,y_train)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=10000, stop_words='english')),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, penalty='none'))])

In [25]:
cvect_pipe.score(X_train, y_train)

0.9946666666666667

In [26]:
cvect_pipe.score(X_test, y_test)

0.636

In [29]:
# pulling coefs out of pipe

coefs = cvect_pipe['logisticregression'].coef_

word_coefs = pd.DataFrame({'words': cvect_pipe['countvectorizer'].get_feature_names_out(), 
              'coefs': coefs[0]})

word_coefs.sort_values(by='coefs', ascending=True).head(15)

,words,coefs
1193,question,-188.030528
1216,rating,-181.721279
955,missed,-171.489736
194,beautiful,-158.545352
1483,tactics,-154.208119
229,books,-153.180771
527,equal,-152.586534
1187,puzzles,-148.289635
842,learning,-137.742809
33,2017,-136.248210


In [30]:
word_coefs.sort_values(by='coefs', ascending=False).head(15)

,words,coefs
961,mom,194.095630
1367,simulated,187.982693
918,mates,154.951895
62,50,154.911009
85,according,151.499170
657,going,149.290185
884,loses,146.612702
1413,spot,146.407533
762,incredible,141.371741
1523,tinder,137.664103


# Logistic Regression with Ngram

In [94]:
# trying N-gram
cvect_pipe_ngram = make_pipeline(CountVectorizer(stop_words = 'english', max_features = 1000, ngram_range=(1,2)),
                           LogisticRegression(max_iter=1000, penalty="none"))

cvect_pipe_ngram.fit(X_train,y_train)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=1000, ngram_range=(1, 2),
                                 stop_words='english')),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, penalty='none'))])

In [95]:
cvect_pipe_ngram.score(X_train, y_train)

0.968

In [96]:
cvect_pipe_ngram.score(X_test, y_test)

0.636

# Naive Bayes

In [143]:
cvect_pipe2 = make_pipeline(CountVectorizer(stop_words = 'english', max_features = 1000),
                           MultinomialNB())

cvect_pipe2.fit(X_train,y_train)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=1000, stop_words='english')),
                ('multinomialnb', MultinomialNB())])

In [144]:
cvect_pipe2.score(X_train, y_train)

0.88

In [145]:
cvect_pipe2.score(X_test, y_test)

0.632

# Random Forest

In [140]:
forest_pipe = make_pipeline(CountVectorizer(stop_words = 'english', max_features = 5000, ngram_range = (1,3)),
                           RandomForestClassifier(n_estimators = 1000, random_state = 123))

forest_pipe.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=5000, ngram_range=(1, 3),
                                 stop_words='english')),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=1000, random_state=123))])

In [141]:
forest_pipe.score(X_train, y_train)

0.9933333333333333

In [142]:
forest_pipe.score(X_test, y_test)

0.664